In [1]:
import os
import numpy as np
import pandas as pd
import csv

In [8]:
datapath = ('data/0/1-s2.0-S0092867422015197-mmc2.csv')
data = pd.read_csv(datapath)
print (data.head())

    genome_id     contig_id    start      end
0  AG-311-I09  AG-311-I09_c   278697   291980
1  AG-311-I09  AG-311-I09_c   357521   402907
2  AG-311-I09  AG-311-I09_c   658277   702729
3  AG-311-I09  AG-311-I09_c   765104   871496
4  AG-311-I09  AG-311-I09_c  1088878  1101236


In [90]:
filtered_df = data[data['genome_id'] == 'MED4']

# Print the filtered rows
display(filtered_df)
filt = filtered_df[['start','end']]
display (filt)
display (filt.loc[[871]])
indices = filt.index

,genome_id,contig_id,start,end
871,MED4,MED4_1,323821,368220
872,MED4,MED4_1,1306399,1367064
873,MED4,MED4_1,620091,628378
874,MED4,MED4_1,653240,702447
875,MED4,MED4_1,762769,774459
876,MED4,MED4_1,804368,826283
877,MED4,MED4_1,914516,973394
878,MED4,MED4_1,979511,984734
879,MED4,MED4_1,1060080,1087933
880,MED4,MED4_1,1152143,1214874


,start,end
871,323821,368220
872,1306399,1367064
873,620091,628378
874,653240,702447
875,762769,774459
876,804368,826283
877,914516,973394
878,979511,984734
879,1060080,1087933
880,1152143,1214874


,start,end
871,323821,368220


In [39]:
datapath2 = ('data/0/2606217259.gff')
data2 = pd.read_csv(datapath2, sep='\t')
print (data2.columns)

Index(['##gff-version 3'], dtype='object')


In [110]:
def listToString(s):
 
    # initialize an empty string
    str1 = " "
 
    # return string
    return (str1.join(s))
     
     
# Driver code
s = ['Geeks', 'for', 'Geeks']
print(listToString(s))

Geeks for Geeks


In [69]:
start_point = 323821
end_point = 368220
island_ids = []
with open(datapath2, 'r') as file:
    for line in file:
        # print (line)
        parts = line.strip().split()
        if len(parts) > 3:
            # print(parts[4])
            if int(parts[3]) >= start_point:
                if int(parts[3]) <= end_point:
                    island_ids.append(parts[8].split('=')[1].split(';')[0])
print (island_ids)

['2606840304', '2606840305', '2606840306', '2606840307', '2606840308', '2606840309', '2606840310', '2606840311', '2606840312', '2606840313', '2606840314', '2606840315', '2606840316', '2606840317', '2606840318', '2606840319', '2606840320', '2606840321', '2606840322', '2606840323', '2606840324', '2606840325', '2606840326', '2606840327', '2606840328', '2606840329', '2606840330', '2606840331', '2606840332', '2606840333', '2606840334', '2606840335', '2606840336', '2606840337', '2606840338', '2606840339', '2606840340', '2606840341', '2606840342', '2606840343', '2606840344', '2606840345', '2606840346', '2606840347', '2606840348', '2606840349', '2606840350', '2606840351', '2606840352', '2606840353']


In [ ]:
start_points = filtered_df[['start']]  # Example series of start points
end_points = filtered_df[['end']] # Example series of end points
start_points = start_points.values.tolist()
end_points = end_points.values.tolist()

island_ids = []

# Open and read the file line by line
with open(datapath2, 'r') as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) > 3:
            start = int(parts[3])
            end = int(parts[4])
            
            # Check if the start and end fall within any of the specified ranges
            for start_point, end_point in zip(start_points, end_points):
                print ((start_point))
                # if start >= int(start_point):
                #     print (start)
# and end <= end_point:
                #     island_ids.append(parts[8].split('=')[1].split(';')[0])
                #     break  # Break the loop once a match is found

print(island_ids)